# Regularization - Ridge Regression

Acest Ridge Regression este o tehnică de regularizare prin care se reduce potențialul de Overfitting la datele de antrenare (overfittinh = high variance). Acest lucru se face prin adăugarea unui termen de penalizare asupra erorii bazate pe valoarea ridicată la pătrat a coeficienților. Pentru a înțelege cum funcționează acest Ridge Regresssion trebuie să stabilim anumiți termeni

Să ne amintim care era formula pentru regresia unei linii (Linear Regression)

ŷ = β0 + β1X1 + β2X2 + ... + βpXp

Acești coeficienți sunt calculați prin minimizarea sumei valorilor ridicate la pătrat. Formula de mai sus poate fi scirsă și sub următoarea formă:

<img src='../SS/rmse.png'>

Din moment ce știm valoare pentru ŷ putem să modificăm acea parte din ecuație

(yi - ŷi)2 = (yi - β0 + β1X1 + β2X2 + ... + βpXp)2

Scopul pentru Ridge Regression este să minimizeze partea de overfitting prin adăugarea unui nou termen de penalizare. Noua formulă arată în acest fel, unde partea du chenarul roșu reprezintă acel termen de penalizare adițional. Ce este interesant în legătură cu acest nou parametru este bazat pe valoarea ridicată la pătrat a coeficienților. Suma acelor coeficienți la pătrat o înmulțim cu o anumită valoare denumită lambda. Acest parametru de lambda o să ne arate cât de mult o să influenșeze acest nou termen de penalizare a erorii și poate fi orice valoare între 0 și infinit

<img src='../SS/ridge.png'>

Ce valoare ar trebuie să alegem pentru acel coeficient lambda? Tocmai ce am spus că poate avea valori între 0 și infinit, ceea ce reprezintă un range mare. Pentru a alege cea mai bună valoare pentru acest parametru o să ne folosim de conceptul de cross validation. O să explorăm mai multe valori pentru coeficientul lambda și o să vedem care perfrormează cel mai bine pe baza unor metrici (MEA, RMSE)

Înainte de a începe să implementăm acest Ridge Regression în Python și Scikit-Learn. În cadrul clasei respective, termenul de lambda este prezent sub denumirea de 'alpha'. Motivul pentru care este alpha este faptul că în cadrul modelelor din Scikit-Learn hyperparamerul care se poate modifica poartă denumirea de alpha

Cu ajutorul lui Scikit-Learn o să realizăm și partea de Cross Validation pentru acest tip de Ridge Regression, iar ce anume folosește Scikit-Learn pentru a defini cel mai bun parametrul din cadrul unui Cross Validation poartă denumirea de 'scorer object'. Convenția din Scikit-Learn este că scorul acela returnat (fie că e precision, recall, f1 score, mearn square error) este mai bun cu cât este mai mare. Atunci când alegem un model în funcție de accuracy, atunci o valoare mare clar este mai bună, dar în cazul unei erori (RMSE, MAE), o valoare mare reprezintă un rezultat mai slab. Pentru a menține această idee de valoarea mai mare reprezintă un model mai bun, Scikit-Learn retunrează o valoare negativă pentru aceste erori de tipul RMSE, MAE.

Acum o să vedem cât este de ușor să realizăm partea de Ridge Regression în Python, utilizând Jupyter Notebook și Scikit Learn. Pentru a ajunge la partea de Ridge Regression, trebuie întâi să facem partea de regularizare de date,

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# reading the file into a DataFrame
df = pd.read_csv('../data/08-Linear-Regression-Models/Advertising.csv')

In [3]:
# printing the head of the DataFrame
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
# splitting the DataFrame into labels and features
X = df.drop('sales', axis=1)
y = df['sales']

In [5]:
# import the Polynomial Feature converter
from sklearn.preprocessing import PolynomialFeatures

In [6]:
# create a polynomial Feature converter
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [7]:
# transform the data
polynomial_features = polynomial_converter.fit_transform(X)

In [8]:
polynomial_features

array([[2.30100000e+02, 3.78000000e+01, 6.92000000e+01, ...,
        9.88757280e+04, 1.81010592e+05, 3.31373888e+05],
       [4.45000000e+01, 3.93000000e+01, 4.51000000e+01, ...,
        6.96564990e+04, 7.99365930e+04, 9.17338510e+04],
       [1.72000000e+01, 4.59000000e+01, 6.93000000e+01, ...,
        1.46001933e+05, 2.20434291e+05, 3.32812557e+05],
       ...,
       [1.77000000e+02, 9.30000000e+00, 6.40000000e+00, ...,
        5.53536000e+02, 3.80928000e+02, 2.62144000e+02],
       [2.83600000e+02, 4.20000000e+01, 6.62000000e+01, ...,
        1.16776800e+05, 1.84062480e+05, 2.90117528e+05],
       [2.32100000e+02, 8.60000000e+00, 8.70000000e+00, ...,
        6.43452000e+02, 6.50934000e+02, 6.58503000e+02]])

In [9]:
# import train-test method
from sklearn.model_selection import train_test_split

In [26]:
# split the data into train-test
X_train, X_test, y_train, y_test = train_test_split(polynomial_features, y, test_size=0.3, random_state=101)

In [27]:
# import the scaler (StandardScaler())
from sklearn.preprocessing import StandardScaler

In [28]:
# create an instance of the scaler
scaler = StandardScaler()

In [29]:
# fit the data to the scaler
scaler.fit(X_train)

StandardScaler()

In [30]:
# transform the data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Până în acest moment doar am pregătit datele pentru Ridge regression. Urmează să creem modelul respectiv. Pentru a crea un model trebuie să importăm din modulul sklearn.linear_model modelul Ridge

In [31]:
# importing the Ridge Regression model
from sklearn.linear_model import Ridge

In [32]:
help(Ridge)

Help on class Ridge in module sklearn.linear_model._ridge:

class Ridge(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidge)
 |  Ridge(alpha=1.0, *, fit_intercept=True, normalize='deprecated', copy_X=True, max_iter=None, tol=0.001, solver='auto', positive=False, random_state=None)
 |  
 |  Linear least squares with l2 regularization.
 |  
 |  Minimizes the objective function::
 |  
 |  ||y - Xw||^2_2 + alpha * ||w||^2_2
 |  
 |  This model solves a regression model where the loss function is
 |  the linear least squares function and regularization is given by
 |  the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
 |  This estimator has built-in support for multi-variate regression
 |  (i.e., when y is a 2d-array of shape (n_samples, n_targets)).
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alpha : {float, ndarray of shape (n_targets,)}, default=1.0
 |      Regularization strength; must be 

Dacă apelăm metoda help pentru modelul de Ridge putem observa că este un parametru denumit 'alpha' (care după cum am spus că face referire la acel coeficient lambda din cadrul Ridge Regression). Default valoarea acestui coeficient este setată la 1.0. O să modificăm valoarea la 10 ca să vedem ce rezultate o să obținem.

In [33]:
# create an instance of the model
ridge_model = Ridge(alpha=10)

In [34]:
# fit the model
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [35]:
# making predictions
y_pred = ridge_model.predict(X_test)

In [36]:
# import the metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [37]:
MAE = mean_absolute_error(y_test, y_pred)

In [38]:
MAE

0.5774404204714177

In [39]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

In [40]:
RMSE

0.894638646131968

Spuneam că acel parametrul de alpha (lambda) poate avea orice valoare cuprinsă între 0 și infinit. Întrebarea care se pune acuma este cum știm dacă valorea 10 este cea mai bună alegere pentru acest parametru? Nu știm asta, iar pentru a răspunde la întrebare trebuie să facem cross validation pentru a verifica mai multe valori pentru parametrul de alpha. Din moment ce acest procedeu este extrem de întâlnit, Scikit-Learn ne pune la dispoziție un model de Ridge Regression care are și partea de cross-validation, model denumit RidgeCV

In [41]:
# importing the cross-validation Ridge Regression model
from sklearn.linear_model import RidgeCV

Acest model de RidgeCV o să realizeze partea de cross-validation pentru un model simplu de Ridge Regression cu mai multe valori pentru parametrul de alpha și o să returneze înapoi care este valorea care a returnat cel mai bun model (cel mai bun parametru). Modelul în sine o să returneze valorea medie pentru o anumită eroare. Partea de scoring metrics este puțin diferită în cadrul unui cross-validation deoarece știm că o valoare mai mare reprezintă un model mai bun, dar pentru partea de eroare acest lucru este invers, o eroare mai mică reprezintă un model mai bun. Să apelăm metoda help pentru acest RidgeCV

In [42]:
help(RidgeCV)

Help on class RidgeCV in module sklearn.linear_model._ridge:

class RidgeCV(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidgeCV)
 |  RidgeCV(alphas=(0.1, 1.0, 10.0), *, fit_intercept=True, normalize='deprecated', scoring=None, cv=None, gcv_mode=None, store_cv_values=False, alpha_per_target=False)
 |  
 |  Ridge regression with built-in cross-validation.
 |  
 |  See glossary entry for :term:`cross-validation estimator`.
 |  
 |  By default, it performs efficient Leave-One-Out Cross-Validation.
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alphas : ndarray of shape (n_alphas,), default=(0.1, 1.0, 10.0)
 |      Array of alpha values to try.
 |      Regularization strength; must be a positive float. Regularization
 |      improves the conditioning of the problem and reduces the variance of
 |      the estimates. Larger values specify stronger regularization.
 |      Alpha corresponds to ``1 / (2C)`` in other lin

O primă diferență pe care o putem observa este faptul că parametrul de alpha a fost modificat cu un parametrul de alphas care are acum ca și valoarea o tuplă de valori, default fiind setat la alphas=(0.1, 1.0, 10.0). Un alt parametru care ne interesează este acel scoring, care reprezintă modalitate prin care să se evalueze performața unui model (o să vedem ce putem alege pentru acel parametru). Un alt parametru de interes este cel de 'cv' care ca și valoare are nevoie de un int care să reprezinte numărul de pachete (folds) în care să fie împărțit setul de date pentru a face partea de cross validation (acel K de la cross validation). Dacă nu se setează nicio valoare pentru acest parametru atunci o să funcționeze pe conceptul de Leave-One-Out prin care creează pachete și lasă o singură valoare pentru partea de testare. Prin urmare, dacă avem un număr de 200 de valori în setul de date o să creeze 200 de pachete și o să ruleze algoritmul de Ridge Regression de 200 de ori. Această opțiune este bună atunci când avem să lucrăm cu un set mic de date. Setul nostru de date cu care lucrăm este destul de mic, putem să lăsăm opțiunea de None pentru parametrul de cv. Să creem acum un model de cross validation cu valorea default pentru alphas

In [44]:
# create a RidgeCV instance
ridge_cv_model = RidgeCV(alphas=(0.1, 1, 10))

In [45]:
# fit the model
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

După cum se preciza la secțiunea de cross-validation, aceasta reține o parte din setul de date pentru partea de validare. Din moment ce rulăm cross-validation pentru setul de date X_train, o parte din acel set de date o să fie reținut pentru evaluarea performanței modelului pentru fiecare dintre valorile selectate la parametrul alphas.

Pentru a afla care dintre acele valori din alphas a avut cea mai bună performanță o să accesăm atributul 'alpha_' din modelul ridge_cv_model

In [46]:
# retrieve the best alpha parameter
ridge_cv_model.alpha_

0.1

Valoarea de mai sus ne spune că parametrul de alpha=0.1 a avut cea mai bună performanță dintre cele 3 (alpha=0.1, alpha=1, alpha=10). Ce anume însă a utilizat pentru evaluare performanței astfel încât să zică că valoarea de 0.1 este cea mai bună? Pentru asta e indicat să oferim o valoare pentru parametrul de scoring. De unde însă știm ce valori acceptă acest parametrul de scoring? Pentru asta, Scikit-Learn are un dicționar care stochează toate aceste valori, dicționar care se găsește în sklearn.metrics și poartă denumirea de SCORERS

In [47]:
from sklearn.metrics import SCORERS

In [49]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

Lista de mai sus reprezintă fiecare eroare disponibilă pe care o putem folosi ca și scoring în cadrul procesului de cross-validation. Valorile au fost modificate astfel îmcât ca o valoare care este mai mare să reprezinte un model mai bun, din această cauză se retunrează pentru anumite erori valoarea negativă. Din lista de mai sus să îi spunem modelului de Ridge Regression să returneze ca și scoring 'neg_mean_absolute_error'

In [50]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1, 10), scoring='neg_mean_absolute_error')

După ce am creat modelul putem să îl antrenăm, se va utiliza în continuare ca și un model normal

In [51]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

Ce se poate observa însă de de această dată este faptul că se precizează metoda prin care se validează performanța unui anumit model

In [52]:
ridge_cv_model.alpha_

0.1

In [53]:
y_pred = ridge_cv_model.predict(X_test)

In [54]:
MAE = mean_absolute_error(y_test, y_pred)

In [55]:
MAE

0.42737748843313855

In [56]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

In [57]:
RMSE

0.6180719926906028

Înainte am rulat un model de Rigde Regression care avea valoarea 10 pentru parametrul de alpha. În momentul în care am utilziat cross-validation și am aflat că valoarea mai potrivită pentru parametrul de alpha ar fi 0.1 (dintre valorile 0.1, 1 și 10) se poate observa faptul că MAE și RMSE sunt mai mici în acest moment, ceea ce înseamnă un model mai bun. Putem de asemenea să afișăm și coeficienții pentru acest model

In [58]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

Ce se poate remarca rapid din acești coeficienți este faptul că niciun coeficient nu este aproape de 0 (ceea ce o să se schimbe la partea de Lasso Regression)

## Recapitulare

În cadrul acestui tutorial am învățat următoarele lucruri:

    1.Ce anume reprezintă Ridge Regression și care a fost motivația să existe acest tip de Ridge Regression

    2. De unde să importăm modelul de Ridge Regression

        from sklearn.linear_model import Ridge

    3. Cum să creem o instanță a acestui model

        ridge = Ridge(alpha=10)

    4. Antrenarea, predicțiile și validarea performanței se fac la fel ca la orice model

        from sklearn.metrics import mean_absolute_error, mean_squared_error

        ridge.fit(X_train, y_train)

        y_pred = ridge.predict(X_test)

        MAE = mean_absolute_error(y_test, y_pred)

        RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

    5. De unde să importăm modelul de Ridge Regression care se ocupă și cu partea de cross-validation

        from sklearn.linear_model import RidgeCV

    6. Cum să creem o instanță a acestui model

        ridge_cv_model = RidgeCV(alpas=(0.1, 1, 10))

            # alphas=(0.1, 1, 10) reprezintă valorile pentru parametrul alpha (acel lambda coeficient) pe care o să le teste re rând modelul de Ridge Regression

    7. Cum să aflăm care a fost cea mai bună valoare pentru acel parametru 

        ridge_cv_model.alpha_

    8. Cum să aflăm toate valorile existente ce se pot utiliza pentru partea de scoring la cross-validation (pe ce bază să se facă validarea)

        from sklearn.metrics import SCORERS

        SCORERS.keys()

    9. Cum să utilizăm un tip de scoring la alegere pentru un model de Ridge Regression cu cross-validation

        ridge_cv_model = RidgeCV(alphas=(0.1, 1, 10), scoring='neg_mean_absolute_error')